In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.cm import get_cmap
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D

In [ ]:
# Lecture des données
data = pd.read_csv('data/flandreau_jobst_internationalcurrencies_data.txt', encoding='cp1252', skiprows=9, header=0, sep='\t')

In [ ]:
# Commençons par créer un fichier attributes en ne gardant que les éléments par pays puis supprimant les lignes qui se répètent
attributes = data[["country_A", "gold", "colony", "debtburden", "rlong", "rshort1900", "rshort1890", "rgdp", "rgdpcap", "poldemo", "coverage"]]
# Sort the dataframe to ensure rows with colony=1 come first
attributes = attributes.sort_values(by=["country_A", "colony"], ascending=[True, False])
# Drop duplicates, keeping the first row (colony=1 will be kept if present)
attributes = attributes.drop_duplicates(subset="country_A", keep="first").reset_index(drop=True)
attributes = attributes.rename(columns={"colony": "is_colonized"})

# Récupérons aussi le nombre de colonies
nb_colonies = data.groupby("country_B")["colony"].sum().reset_index()
nb_colonies = nb_colonies.rename(columns={"colony": "has_colonies"})
attributes = nb_colonies.merge(attributes, left_on="country_B", right_on="country_A", how="left")
attributes = attributes.drop(columns=['country_A'])
attributes = attributes.rename(columns={"country_B": "country"})

In [ ]:
attributes.style.hide(axis="index")

# Première approche, sans attributs

In [ ]:
# Création d'un graph orienté avec la variable 1900 (possible de pondérer le graph pour utiliser les 3 ? Ou de faire avec le commerce ?)
graph_change_1900 = nx.DiGraph()

for _, row in data.iterrows():
    if row["quote1900"] == 1:
        graph_change_1900.add_edge(row["country_A"], row["country_B"])

In [ ]:
n = graph_change_1900.number_of_nodes()
L = graph_change_1900.number_of_edges()
density = L/(n*(n-1))
print(f"Nombre de sommets : {n}")
print(f"Nombre d'arêtes : {L}")
print(f"Densité : {density:.4f}")

On voit déjà que le réseau est très peu dense, avec seulement 218 arêtes sur les $45*44=1980$ possibles, ce qui correspond à une densité de 0.11 : on peut déjà en déduire que la majorité des pays ne sont pas connectés sur le marché des changes.

In [ ]:
def truncate_colormap(cmap_name, min_val=0.2, max_val=1.0, n=100):
    cmap = plt.get_cmap(cmap_name)
    new_cmap = LinearSegmentedColormap.from_list(
        f"{cmap_name}_trunc", cmap(np.linspace(min_val, max_val, n))
    )
    return new_cmap

trunc_oranges = truncate_colormap("Oranges", 0.2, 1.0)

# Determine edge colors
edge_colors = []
for u, v in graph_change_1900.edges():
    if graph_change_1900.has_edge(v, u):
        edge_colors.append("forestgreen")  # reciprocal
    else:
        edge_colors.append("skyblue")  # unidirectional

# Compute node colors based on total degree (in-degree + out-degree)
# Would make sense to use the number of foreign currencies traded locally instead
degrees = dict(graph_change_1900.degree())
node_colors = [degrees[node] for node in graph_change_1900.nodes()]

# Draw graph
plt.figure(figsize=(16, 8))
pos = nx.spring_layout(graph_change_1900, seed=43)

nodes = nx.draw_networkx_nodes(
    graph_change_1900,
    pos,
    node_size=600,
    node_color=node_colors,
    cmap=trunc_oranges)

edges = nx.draw_networkx_edges(
    graph_change_1900,
    pos,
    edge_color=edge_colors)

labels = nx.draw_networkx_labels(graph_change_1900, pos, font_size=9, font_color="white")

legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='forestgreen', markersize=9, label='Unidirectional'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='skyblue', markersize=9, label='Bidirectional')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.colorbar(nodes, label="Total Degree")
plt.axis("off")
plt.show()

On peut constater cela visuellement, avec :
- Un marché des changes très intégré entre l'Europe et les Etats-Unis, intégrant une dizaine de pays.
- Un marché des changes secondaires en Asie, intégrant une demi-douzaine de pays.
- Tous les autres pays sont dépendants de quelques places étrangères pour leurs relations de change.

In [ ]:
# Ou avec un graph restreint aux liens bilatéraux
graph_change_1900 = nx.DiGraph()

for _, row in data.iterrows():
    if row["quote1900"] == 1:
        graph_change_1900.add_edge(row["country_A"], row["country_B"])

# Filter for reciprocal edges
reciprocal_edges = [(u, v) for u, v in graph_change_1900.edges() if graph_change_1900.has_edge(v, u)]
# Get nodes involved in reciprocal edges
nodes_with_reciprocal_edges = set(u for u, v in reciprocal_edges) | set(v for u, v in reciprocal_edges)

plt.figure(figsize=(8, 5))
pos = nx.spring_layout(graph_change_1900, seed=42)
graph_filtered = graph_change_1900.subgraph(nodes_with_reciprocal_edges)

nx.draw_networkx_nodes(graph_filtered, pos, node_size=600, node_color="chocolate")
nx.draw_networkx_edges(graph_filtered, pos, edgelist=reciprocal_edges, edge_color="skyblue")
nx.draw_networkx_labels(graph_filtered, pos, font_size=9, font_color="white")

plt.axis("off")
plt.show()

In [ ]:
# On peut essayer de formaliser en recherchant des cliques
undirected_graph = graph_change_1900.to_undirected()
cliques = list(nx.find_cliques(undirected_graph ))
largest_cliques = sorted(cliques, key=len, reverse=True)[:1]
print(f"Plus grande clique : {largest_cliques[0]}")

In [ ]:
# On regarde quelques éléments sur le graph, en particulier indegree et oudegree
# out-degree = nombre de pays étrangers qui ont un marché de la monnaie du pays en question
# in-degree = nombre monnaies étrangères échangées dans le pays en question

out_degree = graph_change_1900.out_degree()
in_degree = graph_change_1900.in_degree()

out_degree_centrality = nx.out_degree_centrality(graph_change_1900)
in_degree_centrality = nx.in_degree_centrality(graph_change_1900)
betweenness_centrality = nx.betweenness_centrality(graph_change_1900)

df = pd.DataFrame({
    "Country": [node for node, _ in out_degree],
    "Out-Degree": [degree for _, degree in out_degree],
    "In-Degree": [degree for _, degree in in_degree],
    "Out-Degree Centrality": [out_degree_centrality.get(node, 0) for node, _ in out_degree],
    "In-Degree Centrality": [in_degree_centrality.get(node, 0) for node, _ in in_degree],
    "Betweenness Centrality": [betweenness_centrality.get(node, 0) for node, _ in out_degree]
})

df["In / Out ratio"] = df["In-Degree Centrality"] / df["Out-Degree Centrality"]

df_sorted = df.sort_values(by="Betweenness Centrality", ascending=False)

df_sorted.style.hide(axis="index").format({
    "Out-Degree": "{:.0f}",
    "In-Degree": "{:.0f}",
    "Out-Degree Centrality": "{:.3f}",
    "In-Degree Centrality": "{:.3f}",
    "Betweenness Centrality": "{:.3f}",
    "Average Centrality": "{:.3f}",
    "In / Out ratio": "{:.3f}"
})

On peut conforter les observations précédents en étudiant la centralité intermédiaire, qui distingue quelques marchés constituant des points de passage obligés : GBR, DEU, FRA

* => Il faudrait comparer avec le volume des échanges entre pays : est-il aussi très faible entre les pays périphériques ?

On peut aussi essayer de contraster les situations des principales places de change en comparant les centralités intérieure et extérieure. Plus le ratio In / Out est élevé pour un pays donné, plus il sert d'intermédiaire de change pour les autres pays, sans qu'eux-mêmes n'utilisent sa propre monnaie ?

Ce qui permet de distinguer les pays coloniaux comme la France, l'Angleterre et l'Espagne, des pays non-coloniaux comme l'Allemagne, les Etats-Unis ou l'Italie ?

# Seconde approche, avec les attributs

In [ ]:
# Ajoutons les attributs
attributesdata = attributes.set_index('country').to_dict('index').items()
graph_change_1900.add_nodes_from(attributesdata)

print(nx.get_node_attributes(graph_change_1900, 'gold'))

In [ ]:
print(f"La réciprocité moyenne vaut {nx.reciprocity(graph_change_1900):.3f}")
print(f"La densité du graph est de {nx.density(graph_change_1900):.3f}")

In [ ]:
# Jetons un oeil à la transitivité
print(f"La transitivité globale est de {nx.average_clustering(graph_change_1900):.3f}")
print(f"La transitivité moyenne est de {nx.transitivity(graph_change_1900):.3f}")

print('La transitivité locale de chaque pays est : ')
for node, coeff in sorted(nx.clustering(graph_change_1900).items(), key=lambda x: x[1]):
    print(f"{node}: {coeff:.3f}")

In [ ]:
# La betweeness centrality
for node, coeff in sorted(nx.betweenness_centrality(graph_change_1900).items(), key=lambda x: x[1], reverse=True):
    print(f"{node}: {coeff:.3f}")
# Intéressant, la GB ressort, comme l'Allemage d'ailleurs, mais la France recule beaucoup. 
# Cela dit, on peut discuter de la pertinence de cet indicateur, qui suit l'orientation des liens

In [ ]:
# Eignevector centrality
for node, coeff in sorted(nx.eigenvector_centrality(graph_change_1900).items(), key=lambda x: x[1], reverse=True):
    print(f"{node}: {coeff:.3f}")
# Rappel, il s'agit du calcul sur le graph des liens entrants seulement, mais en même temps probablement le plus pertinent

In [ ]:
# Une autre possibilité serait de regarder les colonies
# Création d'un graph des colonies 
graph_colony = nx.DiGraph()

for _, row in data.iterrows():
    if row["colony"] == 1:
        graph_colony.add_edge(row["country_A"], row["country_B"])

print(f"Nombre de sommets : {graph_colony.number_of_nodes()}")
print(f"Nombre d'arêtes : {graph_colony.number_of_edges()}")

nx.draw(graph_colony, with_labels=True, node_size=500, font_size=10)
plt.show()

# Bon par contre ça c'est vraiment un tout petit graph, on a très peu de colonies en fait. Ou plus exactement sans doute, peu de colonies qui ont un marché des changes